# 自动求导
假设对$y=2x^Tx$，关于列向量x求导

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
# 在计算梯度之前，需要一个地方来存储梯度
x.requires_grad_(True) # 等价于'x=torch.arange(4.0, requires_grad_=True)'
# 就可以采用下面的代码访问梯度
x.grad # 默认值为None

In [5]:
# 计算y
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [6]:
# 通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
x.grad == 4 * x

tensor([True, True, True, True])

现在来计算x的另一个函数

In [14]:
# 在默认情况下，PyTorch会累计梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [20]:
# 对非标量调用'backward'需要传入一个'gradient'参数，该参数指定微分函数
x.grad.zero_()
y = x * x # 4维
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward() # 一般不对向量或矩阵求导，而是采用sum的方式，因为sum求导就是单位矩阵对结果没有影响
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [27]:
x.grad.zero_()
y = x * x
u = y.detach() # 将反向传播停止到u这个地方，将u看作一个标量而非关于x的函数
z = u * x

z.sum().backward()
x.grad == 3 * x * x

tensor([ True, False, False, False])

In [28]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到变量的梯度

In [33]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a

tensor(True)